In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Getting Started

In this notebook, we will walk through what it takes to build and assess quality of a simple strategy

We will rely heavily on `TDSCoinbaseData`, `TDSTickGenerator`, and `TDSTransactionTracker`

In [2]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

In [3]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='data', notebook_logging=True)

In [4]:
# Set the start and end date
start_date = '20201001'
end_date = '20201231'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR']

In [5]:
# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

In [6]:
# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)

In [7]:
# Get our first tick
tick = tick_gen.get_tick()
print(tick.p.btc_usd)

namespace(close=10783.24, high=10788.5, low=10779.63, open=10779.63, volume=3.50667047)


In [8]:
# Looks at our current holdings
print(trans_tracker.get_holdings())

{'BTC': 1.0}


## Making our first trade

To make a trade, we use the `make_trade()` function on our `TDSTransactionTracker` object

`make_trade()` : takes 4 arguments

`tick` : the current tick object as output by `gen_tick()`

`product` : `"<BASE>-<QUOTE>"`

`side` : either `"buy"` or `"sell"`

`size` : the amount of ***HELD*** currency to trade. For example, if product is `BTC-USD` and `side` is `sell`, `size` is in ***BTC***. If product is `BTC-USD` and `side` is `buy`, `size` is in ***USD*** 

In [9]:
print('TRADE : ')
# Make a trade
print(trans_tracker.make_trade(tick, 'BTC-USD', 'sell', 0.5))
print('HOLDINGS : ')
# Check the updated holdings
print(trans_tracker.get_holdings())

TRADE : 
{'date': '20201001', 'timestamp': 1601510400, 'side': 'sell', 'size': 0.5, 'price': 10783.24, 'product': 'BTC-USD', 'holdings': {'BTC': 0.5, 'USD': 5381.915084}}
HOLDINGS : 
{'BTC': 0.5, 'USD': 5381.915084}


## Making another trade

You will notice that every trade will slightly decrease your total holdings. This is because the fee rate of 0.0018 is applied automatically on every transaction

In [10]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p.btc_usd)
print('TRADE : ')
# Make a trade
# Note that since we are holding USD and buying BTC, size is in USD
print(trans_tracker.make_trade(tick, 'BTC-USD', 'buy', 2000))
print('HOLDINGS : ')
# Check the updated holdings
print(trans_tracker.get_holdings())

TICK :
namespace(close=10796.02, high=10799.99, low=10782.98, open=10783.24, volume=15.46159883)
TRADE : 
{'date': '20201001', 'timestamp': 1601510460, 'side': 'buy', 'size': 0.185185356653108, 'price': 10796.02, 'product': 'BTC-USD', 'holdings': {'BTC': 0.6849199982956682, 'USD': 3381.915084}}
HOLDINGS : 
{'BTC': 0.6849199982956682, 'USD': 3381.915084}


## Making an invalid trade

The `make_trade()` function will check a number of cases to ensure that the trade you are attempting to make is valid. If you attempt to make an invalid trade, the function call will error with a message explaining why the trade is invalid

In this case, we attempted to sell more bitcoin than we had

In [11]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p.btc_usd)
print('HOLDINGS : ')
# Check our holdings
print(trans_tracker.get_holdings())
print('TRADE : ')
# Attempt to make a trade 
print(trans_tracker.make_trade(tick, 'BTC-USD', 'SELL', 0.9))

TICK :
namespace(close=10795.06, high=10798.73, low=10791.83, open=10796.01, volume=17.63006328)
HOLDINGS : 
{'BTC': 0.6849199982956682, 'USD': 3381.915084}
TRADE : 


Exception: INSUFFICIENT FUNDS : attempted to liquidate 0.9 BTC only holding 0.6849199982956682 BTC

## Liquidating a product

In the `make_trade()` function, if `size < 0` then the entirety of the currency will be traded (if there is sufficient market volume)

In [12]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p.btc_usd)
print('TRADE : ')
print(trans_tracker.make_trade(tick, 'BTC-USD', 'SELL', -1))
print('HOLDINGS : ')
print(trans_tracker.get_holdings())

TICK :
namespace(close=10799.99, high=10799.99, low=10793.67, open=10795.06, volume=2.6463716)
TRADE : 
{'date': '20201001', 'timestamp': 1601510580, 'side': 'sell', 'size': 0.6849199982956682, 'price': 10799.99, 'product': 'BTC-USD', 'holdings': {'BTC': 0.0, 'USD': 10765.729383954926}}
HOLDINGS : 
{'BTC': 0.0, 'USD': 10765.729383954926}


In [13]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p.btc_usd)
print('TRADE : ')
print(trans_tracker.make_trade(tick, 'BTC-USD', 'buy', -1))
print('HOLDINGS : ')
print(trans_tracker.get_holdings())

TICK :
namespace(close=10817.42, high=10829.99, low=10799.45, open=10799.45, volume=47.83403845)
TRADE : 
{'date': '20201001', 'timestamp': 1601510640, 'side': 'buy', 'size': 0.9940664196324213, 'price': 10817.42, 'product': 'BTC-USD', 'holdings': {'BTC': 0.9934301405569727, 'USD': 0.0}}
HOLDINGS : 
{'BTC': 0.9934301405569727, 'USD': 0.0}


## Looking at Trades

At any point you can see your trade history by calling `get_trades()`

In [14]:
display(trans_tracker.get_trades())

[{'date': '20201001',
  'timestamp': 1601510400,
  'side': 'sell',
  'size': 0.5,
  'price': 10783.24,
  'product': 'BTC-USD',
  'holdings': {'BTC': 0.5, 'USD': 5381.915084}},
 {'date': '20201001',
  'timestamp': 1601510460,
  'side': 'buy',
  'size': 0.185185356653108,
  'price': 10796.02,
  'product': 'BTC-USD',
  'holdings': {'BTC': 0.6849199982956682, 'USD': 3381.915084}},
 {'date': '20201001',
  'timestamp': 1601510580,
  'side': 'sell',
  'size': 0.6849199982956682,
  'price': 10799.99,
  'product': 'BTC-USD',
  'holdings': {'BTC': 0.0, 'USD': 10765.729383954926}},
 {'date': '20201001',
  'timestamp': 1601510640,
  'side': 'buy',
  'size': 0.9940664196324213,
  'price': 10817.42,
  'product': 'BTC-USD',
  'holdings': {'BTC': 0.9934301405569727, 'USD': 0.0}}]

In [15]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p.btc_usd)
print('TRADE : ')
print(trans_tracker.make_trade(tick, 'BTC-USD', 'sell', 0.5))
print('HOLDINGS : ')
print(trans_tracker.get_holdings())

TICK :
namespace(close=10824.16, high=10825.0, low=10815.13, open=10815.13, volume=1.91609476)
TRADE : 
{'date': '20201001', 'timestamp': 1601510700, 'side': 'sell', 'size': 0.5, 'price': 10824.16, 'product': 'BTC-USD', 'holdings': {'BTC': 0.49343014055697265, 'USD': 5402.338256}}
HOLDINGS : 
{'BTC': 0.49343014055697265, 'USD': 5402.338256}


## Introducing New Products

We now make trades on 2 new products `ETH-USD` and `BTC-EUR`

In [16]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p)
print('TRADE : ')
print(trans_tracker.make_trade(tick, 'ETH-USD', 'buy', 1000.0))
print('HOLDINGS : ')
print(trans_tracker.get_holdings())

TICK :
namespace(btc_eur=namespace(close=9227.32, high=9237.82, low=9227.32, open=9234.8, volume=0.11392955), btc_usd=namespace(close=10817.88, high=10828.37, low=10817.88, open=10822.81, volume=14.50437905), eth_usd=namespace(close=360.98, high=361.06, low=360.98, open=361.05, volume=175.96316713))
TRADE : 
{'date': '20201001', 'timestamp': 1601510760, 'side': 'buy', 'size': 2.7696227773777213, 'price': 360.98, 'product': 'ETH-USD', 'holdings': {'BTC': 0.49343014055697265, 'USD': 4402.338256, 'ETH': 2.765250152363012}}
HOLDINGS : 
{'BTC': 0.49343014055697265, 'USD': 4402.338256, 'ETH': 2.765250152363012}


In [17]:
tick = tick_gen.get_tick()
print('TICK :')
print(tick.p)
print('TRADE : ')
print(trans_tracker.make_trade(tick, 'BTC-EUR', 'sell', 0.05))
print('HOLDINGS : ')
print(trans_tracker.get_holdings())

TICK :
namespace(btc_eur=namespace(close=9223.48, high=9225.42, low=9223.48, open=9223.92, volume=0.12050106), btc_usd=namespace(close=10815.6, high=10818.67, low=10811.73, open=10818.67, volume=18.71285643), eth_usd=namespace(close=360.63, high=360.98, low=360.55, open=360.98, volume=51.46908024))
TRADE : 
{'date': '20201001', 'timestamp': 1601510820, 'side': 'sell', 'size': 0.05, 'price': 9223.48, 'product': 'BTC-EUR', 'holdings': {'BTC': 0.44343014055697266, 'USD': 4402.338256, 'ETH': 2.765250152363012, 'EUR': 460.3438868}}
HOLDINGS : 
{'BTC': 0.44343014055697266, 'USD': 4402.338256, 'ETH': 2.765250152363012, 'EUR': 460.3438868}


In [18]:
import random

tick = tick_gen.get_tick()
holding_btc = True
## For the rest of the trading period we will randomly flip our BTC and USD holdings
while tick is not None:
    
    # 10% chance of attempting to trade
    if random.uniform(0,1) <= 0.1:
        if holding_btc:
            # Check the max amount of BTC we can trade my multiplying the volume by the max_taken_volume (0.5 in our case)
            available_btc = tick.p.btc_usd.volume * trans_tracker.get_max_taken_volume()
            my_btc = trans_tracker.get_holdings()['BTC']
            # Only trade if valid
            if available_btc >= my_btc:
                trans_tracker.make_trade(tick, 'BTC-USD', 'sell', -1)
                holding_btc = False
        else:
            # Check the available USD by doing the same operation as above and multiplying by the close price since that is the price that we will execute at
            available_usd = tick.p.btc_usd.volume * trans_tracker.get_max_taken_volume() * tick.p.btc_usd.close
            my_usd = trans_tracker.get_holdings()['USD']
            # Only trade if valid
            if available_usd >= my_usd:
                trans_tracker.make_trade(tick, 'BTC-USD', 'buy', -1)
                holding_btc = True
    
    # Get the next tick
    tick = tick_gen.get_tick()

## Visualizing Our Returns

Once we have completed trading, we can visualize our returns by calling `plot_btc_holdings()`

In [19]:
trans_tracker.plot_btc_holdings()

KeyboardInterrupt: 

## Looking at Daily Stats

We can call `get_pct_change_per_day()` to get a dataframe of our daily `BTC` holdings and day to day percent change

`BTC` holding for a single day is derived by taking the shortest path through each product to `BTC` using the last transaction price of a given day

EX:
To get the `BTC` value of an `ETH` holding, we traverse `ETH` -> `USD` -> `BTC`

This logic is all handled for you as a part of `TDSTransactionTracker`

In [20]:
display(trans_tracker.get_pct_change_per_day())

,date,BTC,pct_diff
0,20201001,0.829108,-0.170892
1,20201002,0.702294,-0.152952
2,20201003,0.630000,-0.102939
3,20201004,0.537633,-0.146614
4,20201005,0.454940,-0.153811
...,...,...,...
87,20201227,0.093548,0.066443
88,20201228,0.095426,0.020067
89,20201229,0.094541,-0.009270
90,20201230,0.091634,-0.030746


## Compute Sharpe Ratio

To get out Sharpe ration, we simply call `get_sharpe_ratio()`

This Sharpe ratio is based on the EOD BTC holdings as shown above and uses the BlockFi 1 BTC interest rate as the risk free return

https://blockfi.com/crypto-interest-account

In [21]:
print(trans_tracker.get_sharpe_ratio())

RISK FREE DAILY RETURN : 
0.00016438356164383562
ACTUAL DAILY RETURN : 
-0.024572212328816654
EXCESS STD : 
0.049256698748459345
-0.5021975998997336


## Saving Your Results

To save your trades to a json file in the required format, when you are done trading, simply call `dump_trades("<filepath>")`

In [ ]:
trans_tracker.dump_trades('example_trades.json')